<a href="https://colab.research.google.com/github/garv4gaurav/PySpark/blob/main/PySpark1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 72 kB/s 
     |████████████████████████████████| 198 kB 64.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=e7367b4e335c89903132665b9e382dc2134e02102eb14a44993d3a05dee2944b
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
database_path = "/content/drive/My Drive/DBs/test_Spark/"

In [4]:
import pyspark
import pandas as pd

In [5]:
pd.read_csv(database_path+"spark_test.csv")

,Name,Age,Experience,Salary
0,Gaurav,30,7,1600000
1,varun,30,8,3000000
2,Akash,29,6,10000000
3,Souvik,28,5,700000
4,Ashit,29,5,1200000


In [6]:
from pyspark.sql import SparkSession

In [7]:
sess = SparkSession.builder.appName("Test1").getOrCreate()

In [8]:
sess

In [9]:
df_spark = sess.read.csv(database_path+"spark_test.csv")

In [10]:
df_spark = sess.read.option("header","true").csv(database_path+"spark_test.csv", inferSchema=True)
df_spark = sess.read.csv(database_path+"spark_test.csv", header=True, inferSchema=True)
# Both ways are correct, "inferSchema" is used to fetch correct data type for the columns

In [11]:
df_spark.show()

+------+---+----------+--------+
|  Name|Age|Experience|  Salary|
+------+---+----------+--------+
|Gaurav| 30|         7| 1600000|
| varun| 30|         8| 3000000|
| Akash| 29|         6|10000000|
|Souvik| 28|         5|  700000|
| Ashit| 29|         5| 1200000|
+------+---+----------+--------+



In [12]:
df_spark

DataFrame[Name: string, Age: int, Experience: int, Salary: int]

In [13]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [14]:
df_spark.select("Name").show()

+------+
|  Name|
+------+
|Gaurav|
| varun|
| Akash|
|Souvik|
| Ashit|
+------+



In [15]:
df_spark.dtypes

[('Name', 'string'), ('Age', 'int'), ('Experience', 'int'), ('Salary', 'int')]

In [16]:
df_spark.describe().show()

+-------+-----+------------------+------------------+-----------------+
|summary| Name|               Age|        Experience|           Salary|
+-------+-----+------------------+------------------+-----------------+
|  count|    5|                 5|                 5|                5|
|   mean| null|              29.2|               6.2|        3300000.0|
| stddev| null|0.8366600265340759|1.3038404810405297|3841874.542459709|
|    min|Akash|                28|                 5|           700000|
|    max|varun|                30|                 8|         10000000|
+-------+-----+------------------+------------------+-----------------+



In [17]:
# adding a column in pysspark dataframe
df_spark = df_spark.withColumn("Age after 2 years", df_spark["Age"]+2)
df_spark.show()

+------+---+----------+--------+-----------------+
|  Name|Age|Experience|  Salary|Age after 2 years|
+------+---+----------+--------+-----------------+
|Gaurav| 30|         7| 1600000|               32|
| varun| 30|         8| 3000000|               32|
| Akash| 29|         6|10000000|               31|
|Souvik| 28|         5|  700000|               30|
| Ashit| 29|         5| 1200000|               31|
+------+---+----------+--------+-----------------+



In [18]:
# Drop a column
df_spark = df_spark.drop("Age after 2 years")
df_spark.show()

+------+---+----------+--------+
|  Name|Age|Experience|  Salary|
+------+---+----------+--------+
|Gaurav| 30|         7| 1600000|
| varun| 30|         8| 3000000|
| Akash| 29|         6|10000000|
|Souvik| 28|         5|  700000|
| Ashit| 29|         5| 1200000|
+------+---+----------+--------+



In [19]:
# Rename column name
df_spark = df_spark.withColumnRenamed("Name", "NAME")
df_spark.show()

+------+---+----------+--------+
|  NAME|Age|Experience|  Salary|
+------+---+----------+--------+
|Gaurav| 30|         7| 1600000|
| varun| 30|         8| 3000000|
| Akash| 29|         6|10000000|
|Souvik| 28|         5|  700000|
| Ashit| 29|         5| 1200000|
+------+---+----------+--------+



**HANDLE MISSING VALUES**

In [20]:
sess2 = SparkSession.builder.appName("missing").getOrCreate()
missing_filename = "spark_test_missing.csv"

In [21]:
df_spark2 = sess2.read.load(database_path+"spark_test_missing.csv", format="csv", header="true", sep="\t", inferSchema="true" )
#df_spark2 = sess2.read.format("csv").options(header="true").load(database_path+"spark_test_missing.csv")
df_spark2.show()

+------+----+----------+--------+
|  Name| Age|Experience|  Salary|
+------+----+----------+--------+
|Gaurav|  30|         7| 1600000|
| varun|  30|         8| 3000000|
| Akash|  29|         6|10000000|
|Souvik|null|         5|  700000|
|   Sid|  30|      null|    null|
| Ashit|  29|         5| 1200000|
|  null|  31|         9| 1800000|
+------+----+----------+--------+



In [22]:
df_spark2.columns

['Name', 'Age', 'Experience', 'Salary']

In [23]:
# Drop all null values
df_spark2.na.drop(how="any").show() # this will delete all rows where at least one null value is present

+------+---+----------+--------+
|  Name|Age|Experience|  Salary|
+------+---+----------+--------+
|Gaurav| 30|         7| 1600000|
| varun| 30|         8| 3000000|
| Akash| 29|         6|10000000|
| Ashit| 29|         5| 1200000|
+------+---+----------+--------+



In [24]:
df_spark2.na.drop(how="any", thresh=2).show() # this will delete all rows where at least 2 not null values are present

+------+----+----------+--------+
|  Name| Age|Experience|  Salary|
+------+----+----------+--------+
|Gaurav|  30|         7| 1600000|
| varun|  30|         8| 3000000|
| Akash|  29|         6|10000000|
|Souvik|null|         5|  700000|
|   Sid|  30|      null|    null|
| Ashit|  29|         5| 1200000|
|  null|  31|         9| 1800000|
+------+----+----------+--------+



In [25]:
df_spark2.na.drop(how="any", subset=["Name"]).show() # this will delete all rows for column="Name", where null values are present

+------+----+----------+--------+
|  Name| Age|Experience|  Salary|
+------+----+----------+--------+
|Gaurav|  30|         7| 1600000|
| varun|  30|         8| 3000000|
| Akash|  29|         6|10000000|
|Souvik|null|         5|  700000|
|   Sid|  30|      null|    null|
| Ashit|  29|         5| 1200000|
+------+----+----------+--------+



In [26]:
## FILLING THE MISSING VALUES
df_spark2.na.fill("NA", subset=["Name"]).show()  # Null values are replaced/filled by "NA"

+------+----+----------+--------+
|  Name| Age|Experience|  Salary|
+------+----+----------+--------+
|Gaurav|  30|         7| 1600000|
| varun|  30|         8| 3000000|
| Akash|  29|         6|10000000|
|Souvik|null|         5|  700000|
|   Sid|  30|      null|    null|
| Ashit|  29|         5| 1200000|
|    NA|  31|         9| 1800000|
+------+----+----------+--------+



In [27]:
# Imputer , this Class is used to replace all null values with mean, median or mode
from pyspark.ml.feature import Imputer

imputer = Imputer(
          inputCols=["Age","Experience","Salary"],
          outputCols=["{}_imputed".format(c) for c in ["Age", "Experience", "Salary"]]
        ).setStrategy("mean")

In [28]:
imputer.fit(df_spark2).transform(df_spark2).show()

+------+----+----------+--------+-----------+------------------+--------------+
|  Name| Age|Experience|  Salary|Age_imputed|Experience_imputed|Salary_imputed|
+------+----+----------+--------+-----------+------------------+--------------+
|Gaurav|  30|         7| 1600000|         30|                 7|       1600000|
| varun|  30|         8| 3000000|         30|                 8|       3000000|
| Akash|  29|         6|10000000|         29|                 6|      10000000|
|Souvik|null|         5|  700000|         29|                 5|        700000|
|   Sid|  30|      null|    null|         30|                 6|       3050000|
| Ashit|  29|         5| 1200000|         29|                 5|       1200000|
|  null|  31|         9| 1800000|         31|                 9|       1800000|
+------+----+----------+--------+-----------+------------------+--------------+



**FILTER OPERATIONS**

In [29]:
df_spark2.filter("Salary<=1800000").show()

+------+----+----------+-------+
|  Name| Age|Experience| Salary|
+------+----+----------+-------+
|Gaurav|  30|         7|1600000|
|Souvik|null|         5| 700000|
| Ashit|  29|         5|1200000|
|  null|  31|         9|1800000|
+------+----+----------+-------+



In [30]:
df_spark2.filter("Salary<=1800000").select(["Name","Age"]).show()

+------+----+
|  Name| Age|
+------+----+
|Gaurav|  30|
|Souvik|null|
| Ashit|  29|
|  null|  31|
+------+----+



In [31]:
# Inverse operation
df_spark2.filter( ~(df_spark2["Salary"]<=1800000)).show()

+-----+---+----------+--------+
| Name|Age|Experience|  Salary|
+-----+---+----------+--------+
|varun| 30|         8| 3000000|
|Akash| 29|         6|10000000|
+-----+---+----------+--------+



In [32]:
df_spark2.filter( (df_spark2["Salary"]<=1800000) & 
                  (df_spark2["Age"]<31)
                ).show()

+------+---+----------+-------+
|  Name|Age|Experience| Salary|
+------+---+----------+-------+
|Gaurav| 30|         7|1600000|
| Ashit| 29|         5|1200000|
+------+---+----------+-------+



In [33]:
df_spark3 = sess2.read.load(database_path+"spark_groupby.csv", format="csv", header="true", inferSchema="true", sep=",")
df_spark3.show()

+-------+---------+--------+
|   Name|      Org|  Salary|
+-------+---------+--------+
| Gaurav|Accenture| 1600000|
|  varun|Accenture| 3000000|
|  Akash|     EPIC|10000000|
| Souvik|Accenture|  800000|
|  Ashit|      GOV| 1200000|
|Prakhar|      GOV| 1500000|
|  Karan|     EPIC| 1300000|
+-------+---------+--------+



In [34]:
df_spark3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Org: string (nullable = true)
 |-- Salary: integer (nullable = true)



In [36]:
df_spark3.groupBy("Org").sum().show()

+---------+-----------+
|      Org|sum(Salary)|
+---------+-----------+
|Accenture|    5400000|
|     EPIC|   10000000|
|      GOV|    1200000|
| personal|      70000|
+---------+-----------+



In [37]:
df_spark3.groupBy("Org").count().show()

+---------+-----+
|      Org|count|
+---------+-----+
|Accenture|    3|
|     EPIC|    1|
|      GOV|    1|
| personal|    2|
+---------+-----+



In [38]:
df_spark3.groupBy("Org").mean().show()

+---------+-----------+
|      Org|avg(Salary)|
+---------+-----------+
|Accenture|  1800000.0|
|     EPIC|      1.0E7|
|      GOV|  1200000.0|
| personal|    35000.0|
+---------+-----------+

